In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *
import tensorflow_datasets as tfds
from tfds_data.aff_nist import AffNist
%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm
from distill.repsim_util import get_reps

[nltk_data] Downloading package punkt to /home/dehghani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
databuilder = AffNist(data_dir='../data')
databuilder.download_and_prepare(download_dir='tmp/',
                                    download_config=tfds.download.DownloadConfig(register_checksums=True))

../tmp/training_batches/1.mat
(50000, 40, 40)
Shuffling and writing examples to ../data/aff_nist/0.1.0.incompleteKA6ZCS/aff_nist-train.tfrecord


../tmp/validation.mat
(320000, 40, 40)
Shuffling and writing examples to ../data/aff_nist/0.1.0.incompleteKA6ZCS/aff_nist-validation.tfrecord


../tmp/test.mat
(320000, 40, 40)
Shuffling and writing examples to ../data/aff_nist/0.1.0.incompleteKA6ZCS/aff_nist-test.tfrecord



Dataset aff_nist downloaded and prepared to ../data/aff_nist/0.1.0. Subsequent calls will reuse this data.


In [ ]:
chkpt_dir='../tf_ckpts'
task_name='affnist'
task = TASKS[task_name](get_task_params(), data_dir='../data')

In [ ]:
hparams=get_model_params(task, 'cl_vff', 'ff_svhn2')
print(hparams)

In [ ]:
config={'exp_name':'test',
    'model_config':'ff_svhn2',
    'task_name':'affnist',
    'model_name':'cl_vff',
    'chkpt_dir':'../tf_ckpts',
    'learning_rate': 0.001
    }

task = TASKS[task_name](get_task_params(), data_dir='../data')
model, _ = get_model(config, task, hparams, 0)


In [ ]:
for x,y in task.train_dataset:
    print(x.shape, y.shape)
    break
model(inputs=x, training=True)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=task.get_loss_fn(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=task.get_loss_fn(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

history = model.fit(task.train_dataset,
                  epochs=30,
                  steps_per_epoch=task.n_train_batches,
                  validation_steps=task.n_valid_batches,
                  validation_data=task.valid_dataset,
                  verbose=2
                  )



In [ ]:
model.evaluate(task.test_dataset, steps=task.n_test_batches)

In [ ]:
output = tf.pad(input, [[0, 0], [6,6], [6,6], [0,0]])


In [ ]:
databuilder = tfds.builder("mnist")